In [2]:
import pandas as pd
import numpy as np
import os
import scipy
import statsmodels
from tqdm import tqdm

In [3]:
mypath = os.getcwd()

task_1_path = os.path.join(mypath, 'Задание 1')
task_2_path = os.path.join(mypath, 'Задание 2')

task_1_files = [os.path.join(task_1_path, i) for i in os.listdir(task_1_path)]
task_2_files = [os.path.join(task_2_path, i) for i in os.listdir(task_2_path)]

## Задание 1

**Контекст**

Регулятор постановил, что текущий скрипт диалога с клиентом слишком жёсткий и нужно придумать новый. Нужно проверить, ухудшает ли новый скрипт показатели.

**Тест**

Было проведено тестирование в котором поток клиентов был разбит на две части, где одна часть получила один вариант скрипта, а вторая часть другой вариант, теперь необходимо найти лучший вариант.

**Статистические вводные**

Уровень значимости 5%. Мы хотели бы различать 8%-ное изменение целевой метрики с вероятностью 80%.

In [ ]:
pd.read_csv(task_1_files[1]).head()

,ID,Флаг дозвона,Флаг продажи,Расходы,PV,NPV
0,0,1,0,90,0,-90
1,1,0,0,5,0,-5
2,2,0,0,68,0,-68
3,3,1,0,22,0,-22
4,4,1,0,22,0,-22


Целевая метрика - конверсия в покупку.

$H_0: p_1 = p_2.$ Конверсии в двух группах не имеют значимого различия \
$H_1: p_1 \neq p_2.$ Есть значимое различие в конверсиях \
Для подсчета минимальной необходимой выборки нужно воспользоваться формулой:\
$N = \frac{p(1 - p)(z_{1-\alpha} - z_{\beta})^2}{k \Delta^2}$ \
$N = \frac{p(1 - p)(z_{1-\alpha} - z_{\beta})^2}{(1 - k) \Delta^2}$
Где k - доля теста (контроля), $\Delta$ - MDE, $z_{\alpha}$ - квантиль стандартного нормального распределения, $\sigma^2$ - верхняя оценка дисперсий, P - Базовая конверсия

In [ ]:
p = pd.read_csv(task_1_files[1])['Флаг продажи'].mean()
z_a = scipy.stats.norm.ppf(1 - 0.05/2)
z_b = scipy.stats.norm.ppf(1 - 0.8)
print(f'Значение среднего {p}\nЗначение квантиля 0.975 {z_a}\nЗначение квантиля 0.2 {z_b}')
print('Размер выборки ', p*(1 - p) * (z_a - z_b)**2 / (0.08*p)**2 / (1/2))

Значение среднего 0.3021715330995967
Значение квантиля 0.975 1.959963984540054
Значение квантиля 0.2 -0.8416212335729143
Размер выборки  5664.385861942255


Теперь можно провести z и t тесты. Формулы статистик критериев: \
$z = \frac{\overline{x} - \overline{y}}{\sqrt{\frac{\sigma^2_x}{m} + \frac{\sigma^2_y}{n}}}$ \
$t = \frac{\overline{x} - \overline{y}}{\sqrt{\frac{s^2_x}{m} + \frac{s^2_y}{n}}}$

In [ ]:
cntr = pd.read_csv(task_1_files[2])['Флаг продажи']
test = pd.read_csv(task_1_files[3])['Флаг продажи']
cntr.mean(), test.mean()

(0.2980225988700565, 0.2837217514124294)

In [ ]:
from statsmodels.stats.proportion import proportions_ztest

z_criterion, p_value = statsmodels.stats.proportion.proportions_ztest([sum(cntr), sum(test)],
                                        [len(cntr), len(test)],  alternative='two-sided')
print(f'Количество данных в контроле {len(cntr)}, продаж среди них {sum(cntr)}',
f'\nКоличество данных в тесте {len(test)}, продаж среди них {sum(test)}')
print(f'Z-тест пропорций: значение p_value {p_value} больше 0.05, значит альтернативная гипотеза отвергается')
t_criterion, p_value = scipy.stats.ttest_ind(cntr, test, equal_var = False)
print(f'T-тест: значение p_value {p_value} больше 0.05, значит альтернативная гипотеза отвергается')
print( 'Вычисленный критерий вручную ',
      (cntr.mean() - test.mean()) / np.sqrt(cntr.var() / cntr.size + test.var() / test.size),
      'Значение z-критерия из пакета ', z_criterion)

Количество данных в контроле 5664, продаж среди них 1688 
Количество данных в тесте 5664, продаж среди них 1607
Z-тест пропорций: значение p_value 0.09379789503595067 больше 0.05, значит альтернативная гипотеза отвергается
T-тест: значение p_value 0.0938137897543713 больше 0.05, значит альтернативная гипотеза отвергается
Вычисленный критерий вручную  1.6757550511179125 Значение z-критерия из пакета  1.6756952789890691


Значимых различий в выборках нет, новый скрипт не ухудшит продажи

## Задание 2

**Контекст**

Нынешняя форма авторизации клиента слишком сложная для пользователей, из-за чего часть потенциальных пользователей не подает заявки. Была разработана новая форма, которая должна улучшить утилизацию (пользование) счетов

**Тест**

Поток пользователей был разделен на две части: контроль, где использовался старый метод авторизации и тест, где был использован новый метод авторизации. Нужно понять влияет ли изменение в форме на утилизацию счетов.

**Статистические вводные**

Проверка однородности выбранного параметра осуществяется с уровнем значимости 2%.
Уровень значимости 5%. Мы хотели бы различать 3%-ное изменение целевой метрики с вероятностью 95%.

In [4]:
df = pd.read_csv(task_2_files[0])

In [5]:
df.head()

,ID,Возраст,Доход клиента,Вероятность банкротства,Флаг утилизации счёта,Расходы,PV КК,PV услуги,NPV
0,0,19,21620.835463,0.138061,0,102,0,0,-102
1,1,27,24897.990863,0.035508,1,409,11686,1754,13031
2,2,50,23989.526947,0.098793,0,16,0,0,-16
3,3,18,38442.409756,0.365661,1,788,13738,1578,14528
4,4,24,21291.521612,0.036909,1,1048,6594,2213,7759


Целевая метрика - PV \
$H_0: p_1 = p_2.$ Конверсии в двух группах не имеют значимого различия \
$H_1: p_1 \neq p_2.$ Есть значимое различие в конверсиях \
Сначало нужно посчитать минимальный размер выборки

In [6]:
pv_usl = df['PV услуги']
z_a = scipy.stats.norm.ppf(1 - 0.05)
z_b = scipy.stats.norm.ppf(1 - 0.9)
min_sam = np.round(
    pv_usl.var() * (z_a - z_b) ** 2/ (0.08*pv_usl.mean()) ** 2 / (1/2)
    , 0)
print(f'Минимальный размер выборки {min_sam}')

Минимальный размер выборки 2232.0


Выборки нужно проверить на неоднородность, для этого можно использовать метрику вероятности банкротства

In [7]:
# Проверка неоднородности
cntr = pd.read_csv(task_2_files[1])['Вероятность банкротства']
test = pd.read_csv(task_2_files[2])['Вероятность банкротства']
cntr.mean(), test.mean()

(0.1313060090669901, 0.12926447085414652)

In [8]:
print('Тест Колмогорова-Смирнова на однородность выборок: ', scipy.stats.ks_2samp(cntr, test).pvalue)
print('Тест Андерсона-Дарлинга на однородность выборок: ', scipy.stats.anderson_ksamp([cntr, test]).pvalue)
print('Леман-Розенблатт на однородность выборок: ', scipy.stats.cramervonmises_2samp(cntr, test).pvalue)
print('Выборки однородны')

Тест Колмогорова-Смирнова на однородность выборок:  0.6805095164532771
Тест Андерсона-Дарлинга на однородность выборок:  0.25
Леман-Розенблатт на однородность выборок:  0.49435616760268764
Выборки однородны


<ipython-input-8-1119d11af6bb>:2: UserWarning: p-value capped: true value larger than 0.25
  print('Тест Андерсона-Дарлинга на однородность выборок: ', scipy.stats.anderson_ksamp([cntr, test]).pvalue)


Теперь нужно провести тест сопряженности

In [31]:
cntr = pd.read_csv(task_2_files[1])
test = pd.read_csv(task_2_files[2])

cntr['Группа'] = 'Контроль'
test['Группа'] = 'Тест'
df = pd.concat([cntr, test])

In [33]:
table = pd.crosstab(
    df['Группа'],
    df['Флаг утилизации счёта'],
    margins = True
)
table

Флаг утилизации счёта,0,1,All
Группа,,,
Контроль,587,1645,2232
Тест,562,1670,2232
All,1149,3315,4464


In [35]:
res = scipy.stats.chi2_contingency(table, correction=False)
stat = res.statistic
p_value = res.pvalue

print('Не можем отклонить нулевую гипотезу', '\nХи-квадрат критерий =',stat,'\np-value = ',p_value)

Не можем отклонить нулевую гипотезу 
Хи-квадрат критерий = 0.7324882152097634 
p-value =  0.9472628534686611
